In [67]:
from BCBio import GFF
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
import pandas as pd
from pathlib import Path
from Bio.SeqFeature import SeqFeature, FeatureLocation
import os

# Load datasets and feature files (gff) in pairs
classification_file = Path("../data/classified")
feature_files = Path("../data/features")
datasets = []
output = Path("../data/genomecharts")
output.mkdir(exist_ok=True)

for class_file in classification_file.glob("*.tsv"):
    # Grab dataset name
    set_name = class_file.name.split("_")[0].lower()
    # Special cases for guegler dataset
    if set_name == "guegler" and class_file.name.split("_")[1].lower() == "t4":
        set_name = "gueglert4"
    elif set_name == "guegler" and class_file.name.split("_")[1].lower() == "t7":
        set_name = "gueglert7"

    # Get feature (gff) file
    for feature_file in feature_files.glob("*.gff3"):
        name_split = feature_file.name.split("_")
        set_feature_name = name_split[0].lower()

        if set_name == set_feature_name:
            datasets.append((set_name, class_file, feature_file))

print(f"Found {len(datasets)} datasets.")


def parse_input_files(classification_file, gff_file):
    """
    Parse a gff feature file and classification table.

    Returns classification table as a dataframe and gff as a SeqIO record.
    """
    in_file = gff_file
    class_file = classification_file
    df = pd.read_csv(class_file, sep="\t")
    in_handle = open(in_file)

    for rec in GFF.parse(in_handle):
        record = rec
    in_handle.close()

    return df, record


def draw_genechat(set_name, df, record, feature_basename):
    """
    Draw a gene chart for a genome, classifed in the dataframe, with the features from a seqIO record
    """
    gd_diagram = GenomeDiagram.Diagram(f"{set_name}")
    gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
    gd_feature_set = gd_track_for_features.new_set()

    colormap = {
        "early": "#fee8c8",
        "middle": "#fdbb84",
        "late": "#e34a33",
        "None": "grey",
    }

    # Add genome start marker as a fake feature at position 0
    start_marker = SeqFeature(FeatureLocation(0, 1), type="marker")
    gd_feature_set.add_feature(
        start_marker,
        color="blue",
        label_position="start",
        label_size=10,
    )

    for feature in record.features:
        if feature.type != "gene":
            continue

        classification = df.loc[df["Geneid"] == feature.id, "classification"]
        classification = (
            classification.values[0] if not classification.empty else "None"
        )
        color = colormap[classification]
        gd_feature_set.add_feature(feature, color=color, label=False)

    # Add a separate legend track for legend
    legend_track = gd_diagram.new_track(2, name="Legend", scale=False, height=0.5)
    legend_set = legend_track.new_set()

    # Add a dummy feature per classification as legend
    legend_pos = 0
    c = 800
    for label, color in colormap.items():
        dummy_feature = SeqFeature(
            FeatureLocation(legend_pos, legend_pos + c), type="legend"
        )
        legend_set.add_feature(
            dummy_feature,
            color=color,
            label=True,
            label_size=10,
            name=label,
        )
        legend_pos += c  # space out legend items

    gd_diagram.draw(
        format="circular",
        circular=True,
        pagesize=(20 * cm, 20 * cm),
        start=0,
        end=len(record),
        circle_core=0.7,
    )
    gd_diagram.write(f"{output}/{feature_basename}_genechart.png", "PNG")


# Draw genome chart for each phage in datasets
for name, df_class, gff_file in datasets:
    df, record = parse_input_files(df_class, gff_file)
    feature_basename = gff_file.stem  # get full filename
    draw_genechat(name, df, record, feature_basename)

Found 7 datasets.
